In [1]:
import tensorflow as tf
import numpy as np

In [7]:
char_arr = [c for c in 
    'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [4]:
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [5]:
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [8]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

In [9]:
enc_input = tf.placeholder(tf.float32, 
                           [None, None, n_input])
dec_input = tf.placeholder(tf.float32, 
                           [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

In [10]:
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(
        enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(
        enc_cell, enc_input, dtype=tf.float32)

In [11]:
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(
        dec_cell, output_keep_prob=0.5)

    outputs, dec_states = tf.nn.dynamic_rnn(
        dec_cell, dec_input, initial_state=enc_states, 
        dtype=tf.float32)

In [12]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(
    learning_rate).minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(
    seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run(
        [optimizer, cost], feed_dict={
            enc_input: input_batch, 
            dec_input: output_batch, 
            targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

Epoch: 0001 cost = 3.710475
Epoch: 0002 cost = 2.620276
Epoch: 0003 cost = 1.543407
Epoch: 0004 cost = 1.249719
Epoch: 0005 cost = 0.707627
Epoch: 0006 cost = 0.529583
Epoch: 0007 cost = 0.572436
Epoch: 0008 cost = 0.254951
Epoch: 0009 cost = 0.185563
Epoch: 0010 cost = 0.194928
Epoch: 0011 cost = 0.075562
Epoch: 0012 cost = 0.133786
Epoch: 0013 cost = 0.042282
Epoch: 0014 cost = 0.059581
Epoch: 0015 cost = 0.031583
Epoch: 0016 cost = 0.028749
Epoch: 0017 cost = 0.017789
Epoch: 0018 cost = 0.011733
Epoch: 0019 cost = 0.074032
Epoch: 0020 cost = 0.029123
Epoch: 0021 cost = 0.034889
Epoch: 0022 cost = 0.049170
Epoch: 0023 cost = 0.013903
Epoch: 0024 cost = 0.057146
Epoch: 0025 cost = 0.005182
Epoch: 0026 cost = 0.054996
Epoch: 0027 cost = 0.008042
Epoch: 0028 cost = 0.024208
Epoch: 0029 cost = 0.010874
Epoch: 0030 cost = 0.006040
Epoch: 0031 cost = 0.041981
Epoch: 0032 cost = 0.011818
Epoch: 0033 cost = 0.007656
Epoch: 0034 cost = 0.005764
Epoch: 0035 cost = 0.002764
Epoch: 0036 cost = 0

In [14]:
def translate(word):
    seq_data = [word, 'P' * len(word)]
    i_batch, o_batch, t_batch = make_batch([seq_data])

    prediction = tf.argmax(model, 2)
    result = sess.run(prediction,
                      feed_dict={enc_input: i_batch,
                                 dec_input: o_batch,
                                 targets: t_batch})

    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

In [15]:
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))

word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 소랑
